<a href="https://colab.research.google.com/github/Satyendra0207/Spark_collab/blob/main/Ml_linear_regression_assignment.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install pyspark

     |████████████████████████████████| 281.4 MB 37 kB/s 
     |████████████████████████████████| 198 kB 49.1 MB/s 
  Created wheel for pyspark: filename=pyspark-3.2.1-py2.py3-none-any.whl size=281853642 sha256=c36c28ec093dc8545e2382890f461f2b79ddbf7e18cca74b46d28ec539f0434b
  Stored in directory: /root/.cache/pip/wheels/9f/f5/07/7cd8017084dce4e93e84e92efd1e1d5334db05f2e83bcef74f
Successfully built pyspark


In [2]:
from pyspark.sql import SparkSession
session=SparkSession.builder.appName("classification_project").getOrCreate()

# Regression Problem
Linear Regression


In [3]:
#importing data set 
data=session.read.csv("automobile2.csv",header=True,inferSchema=True)

In [4]:
data.show(5)

+---+-----------+---------+------------+-----------+------------+---------------+----------+------+-----+------+-----------+-----------+----------------+-----------+-----------+-----------------+----------+--------+-----------+-----+
|_c0|       make|fuel_type|num_of_doors| body_style|drive_wheels|engine_location|wheel_base|length|width|height|curb_weight|engine_type|num_of_cylinders|engine_size|fuel_system|compression_ratio|horsepower|city_mpg|highway_mpg|price|
+---+-----------+---------+------------+-----------+------------+---------------+----------+------+-----+------+-----------+-----------+----------------+-----------+-----------+-----------------+----------+--------+-----------+-----+
|  0|alfa-romero|      gas|         two|convertible|         rwd|          front|      88.6| 168.8| 64.1|  48.8|       2548|       dohc|            four|        130|       mpfi|              9.0|       111|      21|         27|13495|
|  1|alfa-romero|      gas|         two|convertible|         rwd

In [5]:
data.columns

['_c0',
 'make',
 'fuel_type',
 'num_of_doors',
 'body_style',
 'drive_wheels',
 'engine_location',
 'wheel_base',
 'length',
 'width',
 'height',
 'curb_weight',
 'engine_type',
 'num_of_cylinders',
 'engine_size',
 'fuel_system',
 'compression_ratio',
 'horsepower',
 'city_mpg',
 'highway_mpg',
 'price']

In [6]:
data.describe()

DataFrame[summary: string, _c0: string, make: string, fuel_type: string, num_of_doors: string, body_style: string, drive_wheels: string, engine_location: string, wheel_base: string, length: string, width: string, height: string, curb_weight: string, engine_type: string, num_of_cylinders: string, engine_size: string, fuel_system: string, compression_ratio: string, horsepower: string, city_mpg: string, highway_mpg: string, price: string]

In [7]:
data.printSchema()

root
 |-- _c0: integer (nullable = true)
 |-- make: string (nullable = true)
 |-- fuel_type: string (nullable = true)
 |-- num_of_doors: string (nullable = true)
 |-- body_style: string (nullable = true)
 |-- drive_wheels: string (nullable = true)
 |-- engine_location: string (nullable = true)
 |-- wheel_base: double (nullable = true)
 |-- length: double (nullable = true)
 |-- width: double (nullable = true)
 |-- height: double (nullable = true)
 |-- curb_weight: integer (nullable = true)
 |-- engine_type: string (nullable = true)
 |-- num_of_cylinders: string (nullable = true)
 |-- engine_size: integer (nullable = true)
 |-- fuel_system: string (nullable = true)
 |-- compression_ratio: double (nullable = true)
 |-- horsepower: integer (nullable = true)
 |-- city_mpg: integer (nullable = true)
 |-- highway_mpg: integer (nullable = true)
 |-- price: integer (nullable = true)



In [17]:
#importing required functions 
from pyspark.ml.feature import VectorAssembler,StringIndexer,OneHotEncoder

##Changing string datatype of attruibutes using StringIndexer
str_idx=StringIndexer(inputCols=['make','fuel_type','num_of_doors','body_style',"num_of_cylinders","drive_wheels"],outputCols=['make_2','fuel_type_2','num_of_doors_2','body_style_2',"num_of_cylinders2","drive_wheels2"],handleInvalid="skip",stringOrderType="frequencyDesc")

In [9]:
#assigning categorical variable into numerical using OnehotEncoder
#onehot=OneHotEncoder(inputCols=["make_2","fuel_type_2","num_of_doors_2","body_style_2","num_of_cylinders2","drive_wheels2"],outputCols=["make2","fuel_type2","num_of_doors2","body_style2","new_num_of_cylinders_2","drive_wheels_2"])
#onehot=OneHotEncoder(inputCols=['make','fuel_type','num_of_doors','body_style',"num_of_cylinders","drive_wheels"],outputCols=["make2","fuel_type2","num_of_doors2","body_style2","new_num_of_cylinders_2","drive_wheels_2"])

In [27]:
#assembling features using Vector Assembler
#vec_assem=VectorAssembler(inputCols=["new_make","new_fuel_type","new_num_of_doors2","new_num_of_cylinders"],outputCol="allfeatures")
vec_assem=VectorAssembler(inputCols=['make_2','fuel_type_2','num_of_doors_2','body_style_2',"num_of_cylinders2","drive_wheels2","wheel_base",'length','width','height','curb_weight','engine_size','horsepower','city_mpg','highway_mpg'],outputCol="allfeatures")

In [28]:
#importing model 
from pyspark.ml.regression import LinearRegression
lr=LinearRegression(featuresCol="allfeatures",labelCol="price")

In [29]:
from pyspark.ml import Pipeline
mypipe=Pipeline(stages=[str_idx,vec_assem,lr])

In [30]:
training,test=data.randomSplit([0.7,0.3])

In [31]:
training.show(5)

+---+-----------+---------+------------+-----------+------------+---------------+----------+------+-----+------+-----------+-----------+----------------+-----------+-----------+-----------------+----------+--------+-----------+-----+
|_c0|       make|fuel_type|num_of_doors| body_style|drive_wheels|engine_location|wheel_base|length|width|height|curb_weight|engine_type|num_of_cylinders|engine_size|fuel_system|compression_ratio|horsepower|city_mpg|highway_mpg|price|
+---+-----------+---------+------------+-----------+------------+---------------+----------+------+-----+------+-----------+-----------+----------------+-----------+-----------+-----------------+----------+--------+-----------+-----+
|  0|alfa-romero|      gas|         two|convertible|         rwd|          front|      88.6| 168.8| 64.1|  48.8|       2548|       dohc|            four|        130|       mpfi|              9.0|       111|      21|         27|13495|
|  1|alfa-romero|      gas|         two|convertible|         rwd

In [32]:
fitmodel=mypipe.fit(training)

In [33]:
result = fitmodel.transform(test)
result.show(5)

+---+---------+---------+------------+----------+------------+---------------+----------+------+-----+------+-----------+-----------+----------------+-----------+-----------+-----------------+----------+--------+-----------+-----+------+-----------+--------------+------------+-----------------+-------------+--------------------+------------------+
|_c0|     make|fuel_type|num_of_doors|body_style|drive_wheels|engine_location|wheel_base|length|width|height|curb_weight|engine_type|num_of_cylinders|engine_size|fuel_system|compression_ratio|horsepower|city_mpg|highway_mpg|price|make_2|fuel_type_2|num_of_doors_2|body_style_2|num_of_cylinders2|drive_wheels2|         allfeatures|        prediction|
+---+---------+---------+------------+----------+------------+---------------+----------+------+-----+------+-----------+-----------+----------------+-----------+-----------+-----------------+----------+--------+-----------+-----+------+-----------+--------------+------------+-----------------+-----

In [34]:
from pyspark.ml.evaluation import RegressionEvaluator
eval_obj=RegressionEvaluator(predictionCol="prediction",labelCol="price",metricName="r2")
accuracy=eval_obj.evaluate(result)
print(accuracy)

0.7588858810626178


# **Classification problem**
Logistic Regression

In [ ]:
df=session.read.csv("tips_dataset.csv",header=True,inferSchema=True)

In [ ]:
df.show(20)

+---+---+---------+----------+-------------+-----------------+--------+----+------+-------------+
|UID| ID|  Tipster|      Date|        Track|            Horse|Bet Type|Odds|Result|TipsterActive|
+---+---+---------+----------+-------------+-----------------+--------+----+------+-------------+
|  1|  1|Tipster A|24/07/2015|        Ascot|        Fredricka|     Win| 8.0|  Lose|         true|
|  2|  2|Tipster A|24/07/2015|       Thirsk|    Spend A Penny|     Win| 4.5|  Lose|         true|
|  3|  3|Tipster A|24/07/2015|         York|Straightothepoint|     Win| 7.0|  Lose|         true|
|  4|  4|Tipster A|24/07/2015|    Newmarket|   Miss Inga Sock|     Win| 5.0|  Lose|         true|
|  5|  5|Tipster A|25/07/2015|        Ascot|            Peril|     Win|4.33|   Win|         true|
|  6|  6|Tipster A|25/07/2015|         York|          Aldreth|     Win| 6.0|  Lose|         true|
|  7|  7|Tipster A|25/07/2015|    Newcastle|     Niceonemyson|     Win| 6.0|  Lose|         true|
|  8|  8|Tipster A|2

In [ ]:
df.describe()

DataFrame[summary: string, UID: string, ID: string, Tipster: string, Date: string, Track: string, Horse: string, Bet Type: string, Odds: string, Result: string]

In [ ]:
df.columns

['UID',
 'ID',
 'Tipster',
 'Date',
 'Track',
 'Horse',
 'Bet Type',
 'Odds',
 'Result',
 'TipsterActive']

In [ ]:
str_idx3=StringIndexer(inputCols=['Tipster','Track','Horse','Bet Type','Odds','Result'],outputCols=['new_Tipster','new_Track','new_Horse','new_Bet Type','new_Odds','new_Result'],handleInvalid='skip')

In [ ]:
onehot1=OneHotEncoder(inputCols=["new_Track","new_Horse","new_Bet Type"],outputCols=["new_Track2","new_Horse2","new_Bet Type2"])

In [ ]:
vect_assem=VectorAssembler(inputCols=["new_Track2","new_Horse2","new_Bet Type2","new_Odds","TipsterActive"],outputCol="allfeatures3")

In [ ]:
from pyspark.ml.classification import LogisticRegression
model=LogisticRegression(featuresCol="allfeatures3",labelCol="new_Result")

In [ ]:
from pyspark.ml import Pipeline
mypipeline=Pipeline(stages=[str_idx3,onehot1,vect_assem,model])

In [ ]:
training,test=df.randomSplit([0.7,0.3])

In [ ]:
fitmodel=mypipeline.fit(training)

In [ ]:
result2=fitmodel.transform(test)
result2.show(8)

+---+---+---------+----------+---------+----------------+--------+----+------+-------------+-----------+---------+---------+------------+--------+----------+----------------+--------------------+-------------+--------------------+--------------------+--------------------+----------+
|UID| ID|  Tipster|      Date|    Track|           Horse|Bet Type|Odds|Result|TipsterActive|new_Tipster|new_Track|new_Horse|new_Bet Type|new_Odds|new_Result|      new_Track2|          new_Horse2|new_Bet Type2|        allfeatures3|       rawPrediction|         probability|prediction|
+---+---+---------+----------+---------+----------------+--------+----+------+-------------+-----------+---------+---------+------------+--------+----------+----------------+--------------------+-------------+--------------------+--------------------+--------------------+----------+
|  4|  4|Tipster A|24/07/2015|Newmarket|  Miss Inga Sock|     Win| 5.0|  Lose|         true|       12.0|      4.0|  10222.0|         0.0|     8.0|  

In [ ]:
from pyspark.ml.evaluation import BinaryClassificationEvaluator
eval_obj2=BinaryClassificationEvaluator(rawPredictionCol="prediction",labelCol='new_Result')
accuracy2=eval_obj2.evaluate(result2)
print(accuracy2)

0.5746556225196223
